# Hosted Agent 배포하기

## Agent 배포 유형

Microsoft Foundry에서 에이전트는 배포 방식에 따라 크게 두 가지 유형으로 나뉩니다.

### 1. 일반적인 Agent (Managed Agent)

표준 Agent는 Foundry의 관리되는 런타임 환경 안에서 동작하는 에이전트입니다. 우리가 Foundry Portal에서 선언적으로 생성하거나 Python, C#과 같은 언어로 Agent를 Foundry에 생성할 경우, 이들은 기본적으로 Foundry가 제공하는 Agent Runtime 위에서 실행됩니다. 이 경우 CPU, 메모리 같은 리소스를 지정할 수 없으며, 서버리스 형태로 동작하는 추상화된 실행 환경이 제공됩니다.

### 2. Hosted Agent
다른 하나는 Hosted Agent인데, 이는 여러분이 개발한 에이전트 코드를 Container 기반으로 패키징하여 Foundry Agent Service에 배포하면 Agent Service가 그를 관리되는 컨테이너 서비스 환경에서 호스팅하는 방식입니다. 개발자는 컨테이너가 실행되는 환경의 CPU, 메모리, replica 수 등을 지정할 수 있으며 Foundry가 컨테이너 인프라를 관리합니다. 이 방식은 특히 더 많은 컴퓨팅 자원이 필요하거나, 특정 라이브러리나 종속성을 포함해야 하는 에이전트 애플리케이션에 적합합니다.

#### Hosted Agent 과금
Hosted Agent는 현재 Preview 상태이기에, 과금에 대한 정책이 명확하지 않습니다. 하지만, 향후에는 리소스 사용량에 따라 과금이 이루어질 것으로 예상됩니다. 그렇기에, 아마도 Hosted Agent는 어느 정도 이상의 컴퓨팅 자원이 필요한 에이전트 애플리케이션에 적합해 보입니다. CPU와 메모리는 1:2 비율로 지정해야 하며, 현재는 최소 (0.25, 0.5Gi) 부터 최대 (3.5, 7Gi) 까지 지정할 수 있을 것으로 보이나, 이는 향후 변경될 수 있습니다.


## Hosted Agent 배포를 위한 준비사항

Hosted Agent를 배포하려면, 먼저 에이전트 코드를 개발하고 이를 Dockerfile을 사용해서 컨테이너 이미지로 패키징해야 합니다. 이 과정은 일반적인 Docker 컨테이너 이미지를 만드는 과정과 유사합니다. 또한, Hosted Agent는 AI 관련된 특정 환경 변수를 필요로 하며, 이들은 `.env` 파일에 정의되어야 합니다. 예를 들면, Azure OpenAI 서비스의 엔드포인트와 배포 이름, Azure AI 프로젝트의 엔드포인트와 모델 배포 이름 등이 필요합니다.

### Hosted Agent를 위한 Azure 리소스
Host Agent는 다음과 같은 Azure 리소스가 필요할 수 있습니다. 

- Microsoft Foundry Project: Foundry의 기본이 되는 프로젝트
- Foundry Model Deployments: 에이전트에서 사용할 AI 모델(예, GPT-4.1)의 배포
- Azure Container Registry: 에이전트 배포를 위한 컨테이너 이미지 저장 및 관리
- Application Insights: 에이전트의 모니터링 및 로깅

물론, 기존에 사용 중인 Azure 서비스들이 존재한다면 그들을 활용할 수 있습니다. 하지만 이번 실습에서는 빠르게 새로 만들면서 진행하도록 하겠습니다. 이를 위해서는 필요한 Azure 리소스를 빠르게 생성하는 데 도움이 되는 Starter kit을 활용할 수 있습니다.

### Microsoft Foundry azd bicep starter kit

Hosted Agent를 위해 필요한 Azure 리소스를 빠르게 생성할 수 있도록 도움을 주는 샘플 프로젝트가 있습니다. 다음 링크를 참고하여, Hosted Agent를 위한 Azure 리소스를 빠르게 생성할 수 있습니다.

[Microsoft Foundry azd bicep starter kit (basic)](https://github.com/Azure-Samples/azd-ai-starter-basic?tab=readme-ov-file)

이 Kit을 사용하면 필요한 인프라를 매우 쉽게 생성할 수 있으며, 다음의 그림과 같은 순서로 Azure 리소스를 생성합니다.

![Microsoft_Foundry_azd_bicep_starter_kit.png](./images/Microsoft_Foundry_azd_bicep_starter_kit.png)

## 배포할 에이전트 개발하기

Hosted Agent를 배포하기 전에 우선 배포할 에이전트가 필요합니다. 이전 실습에서 다룬 Agent 개발 방법을 참고하여, Hosted Agent로 배포할 에이전트를 개발합니다. Hosted Agent는 기본적으로 Foundry의 Managed Agent와 동일한 방식으로 개발됩니다. 다만, Hosted Agent는 컨테이너 환경에서 실행되므로, Dockerfile을 사용하여 컨테이너 이미지를 빌드할 수 있도록 해야 합니다. 

### 에이전트 코드 준비
현재 워크샵 리포에는 이미 샘플로 만들어 둔 HostedAgent 코드가 있습니다. 현 리포에서 `HostedAgent` 디렉토리로 이동하면 개발된 에이전트의 코드와 구조를 살펴볼 수 있습니다. 이는 Microsoft Foundry 오픈소스의 Sample들 중에서 Hosted Agent 예제를 참고하여 만들어졌습니다. 원본 샘플의 링크는 다음과 같습니다.

https://github.com/microsoft-foundry/foundry-samples/tree/main/samples/python/hosted-agents/agent-framework

여러 샘플 중 `agent-with-foundry-tools`를 참고했으며, 이 샘플은 Microsoft Learn MCP 도구를 사용하는 에이전트를 구현한 예제입니다. 

우리는 현재 리포에 있는 `HostedAgent` 디렉토리의 `main.py` 파일을 열어, 에이전트가 어떻게 구현되었는지 살펴봅니다. 이 에이전트는 Microsoft Learn MCP 도구를 사용하여 기술 문서 질문에 답변하는 에이전트입니다. 그렇기에, 이 에이전트가 올바로 동작하기 위해서는 Microsoft Learn MCP 도구와의 연결이 필요합니다. 이 연결은 Azure AI 프로젝트에서 생성할 수 있으며, 연결 ID를 `.env` 파일에 설정해야 합니다. 다음 단계를 따라서 이러한 작업을 진행합니다.

### Microsoft Learn MCP 도구 준비
여러분이 앞선 실습에서 사용했던 Microsoft Foundry 프로젝트를 열고, 다음의 이미지를 따라서 Microsoft Learn MCP 도구와의 연결을 생성합니다. 그리고 연결이 생성되고 나면 마지막 이미지를 참고해서 `프로젝트 연결 ID`를 복사합니다.  

<img src="./images/hosted-agent-tool1.png" width="70%">   

  
<img src="./images/hosted-agent-tool2.png" width="70%">  

  
<img src="./images/hosted-agent-tool3.png" width="70%">  

### 환경 변수 설정

이제 Hosted Agent가 올바로 동작하기 위해 필요한 환경 변수를 설정합니다.
앞서 복사해 둔 `프로젝트 연결 ID`를 `HostedAgent/.env.sample` 파일의 `AZURE_AI_PROJECT_TOOL_CONNECTION_ID` 항목에 붙여넣습니다.  
또한, `AZURE_OPENAI_ENDPOINT`, `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`, `AZURE_AI_PROJECT_ENDPOINT`, `AZURE_AI_MODEL_DEPLOYMENT_NAME` 항목도 여러분의 Azure OpenAI 서비스와 Azure AI 프로젝트에 맞게 수정합니다. 수정이 완료되면, 파일 이름을 `.env`로 변경하여 환경 변수를 설정합니다.

### 로컬에서 에이전트 테스트하기

이제 로컬에서 에이전트를 테스트해봅니다.  
터미널을 열고 현재 가상 환경에서 실행 중인지 확인합니다.   
만일, 가상 환경이 아니라면 다음 명령어를 사용해서 가상 환경을 활성화합니다.  

>```bash
>./.venv/Scripts/Activate.ps1
>```

그 다음, 다음 명령어를 사용해서 필요한 패키지를 설치합니다.

>```bash
>pip install -r ./HostedAgent/requirements.txt
>```

이제, 현재 디렉토리(MAF_Workshop)에서 다음 명령어를 사용해서 에이전트를 실행합니다.

>```bash
>python ./HostedAgent/main.py
>```
이제 터미널에 에이전트가 시작되었다는 메시지가 다음과 유사하게 출력됩니다. 

>```bash
>INFO:     Started server process [20124]
>INFO:     Waiting for application startup.
>```

이제 다음의 Python 코드를 실행하여, 에이전트가 정상적으로 동작하는지 확인합니다.

In [ ]:
import requests
import json

response = requests.post(
    "http://localhost:8088/responses",
    headers={"Content-Type": "application/json"},
    json={"input": "Foundry hosted agent는 어떻게 배포하나요?", "stream": False}
)

print(json.dumps(response.json(), indent=2, ensure_ascii=False))

## Hosted Agent 배포하기

### Azure Developer CLI 설치
- 우선 [Azure Developer CLI version 1.23.0 이상의 버전](https://learn.microsoft.com/en-us/azure/developer/azure-developer-cli/install-azd)을 설치해야 합니다. 링크를 클릭하여 설치 방법을 참고합니다.

Windows의 경우, **winget install microsoft.azd** 명령으로 쉽게 설치할 수 있습니다.

### Starter Kit을 통한 Azure 환경 초기화

이제 Microsoft Foundry azd bicep starter kit을 사용하여 Hosted Agent를 위한 Azure 리소스를 생성할 준비를 하겠습니다.

우선, 새로운 터미널을 열고 다음의 코드를 실행하여 MAF_Workshop 폴더 밑에 `Azure-HostedAgent-Deployment` 폴더를 생성하고 해당 폴더로 이동하여 Microsoft Foundry azd bicep starter kit을 사용하여 initialize 합니다.

```powershell
    mkdir Azure-HostedAgent-Deployment
    cd Azure-HostedAgent-Deployment
    azd init -t https://github.com/Azure-Samples/azd-ai-starter-basic
```
azd를 실행하면 고유한 환경 이름을 물어오는데 그 명칭은 다음과 같이 기입합니다. 

Enter a unique environment name: `hosted-agent-env`

추후 이 명칭으로 리소스 그룹(rg-hosted-agent-env)이 만들어지고, Foundry 프로젝트도 해당 명칭으로 생성됩니다. 새로운 프로젝트가 초기화되면 `Azure-HostedAgent-Deployment` 폴더 안에 Azure 리소스 배포에 필요한 Bicep 템플릿과 설정 파일들을 다운로드되고 초기화됩니다.

### 에이전트 샘플 초기화

이제 터미널에서 다음의 명령을 실행하여 에이전트 샘플을 초기화 합니다.

>```powershell
> azd ai agent init -m https://github.com/taeyo-kim/MAF_Workshop/blob/main/HostedAgent/agent.yaml
>```

그러면, 대화형 명령이 실행되면서 몇 가지 설정 값을 물어보는데 각 항목에 대해 다음과 같이 입력합니다.  

Azure 구독 - Foundry에서 리소스를 생성할 **자신의 Azure 구독을 선택** 합니다.  
Location - **East US 2** 지역을 선택합니다.   
Model SKU - **GlobalStandard** 를 선택합니다.  
Deployment name - **gpt-4.1** 기본값을 선택합니다.  
Container memory - 컨테이너 메모리 값으로 기본값 **2Gi** 를 입력하거나 수락합니다.  
Container CPU - 컨테이너 CPU 값으로 기본값 **1**을 입력하거나 수락합니다.  
Minimum replicas - 컨테이너 최소 복제본(replica) 수로 **1** 을 입력하거나 수락합니다.  
Max replicas - 컨테이너 최대 복제본(replica) 수로 **3** 을 입력하거나 수락합니다.

<img src="./images/azd_init.png" width="80%">

>**주의**
>
> 이러한 명령을 통해서 리소스를 생성하려면, 여러분은 Azure 구독에 대한 **Contributor** 권한이 필요합니다.

### Foundry Agent Service로 배포하기

이제 `azd up` 명령어를 사용하여 실제로 배포를 수행합니다.  
`azd up` 명령은 인프라를 프로비저닝하고, 애플리케이션을 패키징하며, Azure에 배포하는 세 단계를 한 번에 수행합니다.   
터미널을 열고 다음의 명령을 실행합니다.  

```powershell
    azd up
```

이 명령을 실행하면, 약 8~9분 정도 소요되며 다음과 같은 작업들이 순차적으로 수행됩니다(수행 시간은 환경에 따라 다를 수 있습니다).

<img src="./images/deploy_compeleted.png" width="60%">

### Foundry Portal에서 Hosted Agent 확인하기

이제 Microsoft Foundry Portal에 접속하여, Hosted Agent가 올바로 배포되었는지 확인합니다.

1. [Microsoft Foundry Portal](https://ai.azure.com/nextgen)에 접속하여, 앞서 지정한 환경 이름(`hosted-agent-env`)으로 생성된 프로젝트에 접속합니다.   
프로젝트 명은 접두어 **`ai-project-`** 가 붙은 형태이기에, **`ai-project-hosted-agent-env`** 와 같은 형태일 것입니다.   
혹은 Azure Portal에서 해당 리소스 그룹(**rg-hosted-agent-env**)으로 이동한 후, Foundry 프로젝트 링크를 클릭하여 접속할 수도 있습니다.  

2. 좌측 메뉴에서 `Agents` 탭을 클릭하여, 배포된 Hosted Agent가 목록에 나타나는지 확인하고, 해당 에이전트를 클릭합니다.  
에이전트는 유형이 Hosted로 표시되며, 상태가 `Running`으로 나타나야 합니다.

    <img src="./images/hosted-agent1.png" width="70%">

3. 채팅 창에 `what is Microsoft Agent Framework?` 라고 입력하여, 에이전트가 올바로 응답하는지 확인합니다.

    <img src="./images/hosted-agent2.png" width="70%">

4. 채팅 창 우측 상단의 `디버그` 아이콘을 클릭해서, 에이전트의 대화 로그와 도구 호출 내역을 확인합니다.  
도구 호출 내역이 있는지 확인하여, 에이전트가 Microsoft Learn MCP 도구를 올바로 호출하는지 검증합니다.

    <img src="./images/hosted-agent3.png" width="70%">

### 더욱 세밀한 배포가 필요하다면

조금 더 배포를 커스터마이징 해야 할 필요가 있다면, 컨테이너 이미지를 별도로 배포하고 Python SDK를 사용하여 배포 스크립트를 작성할 수도 있습니다. 
자세한 내용은 다음의 공식 문서를 참고하시기 바랍니다.

**Deploy using the Python SDK**

https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/deploy-hosted-agent?view=foundry&tabs=bash#deploy-using-the-python-sdk
